In [11]:
from ucsc_genomes_downloader import Genome
import pandas as pd
import numpy as np

In [7]:
promoters_labels = pd.read_csv('./pre_processed_dataset/labels_promoters.csv', index_col = [0,1,2,3])
enhancers_labels = pd.read_csv('./pre_processed_dataset/labels_enhancers.csv', index_col = [0,1,2,3])
promoters = pd.read_csv('./pre_processed_dataset/epigenomes_promoters.csv', index_col = [0,1,2,3])
enhancers = pd.read_csv('./pre_processed_dataset/epigenomes_enhancers.csv', index_col = [0,1,2,3])

epigenomes = {
    "promoters": promoters,
    "enhancers": enhancers
}

labels = {
    "promoters": promoters_labels,
    "enhancers": enhancers_labels
}

In [2]:
window_size = 256 # nucleotidi

In [3]:
genome = Genome('hg38')

Loading chromosomes for genome hg38:   0%|                                                      | 0/25 [00:00<…

In [8]:
def to_bed(data:pd.DataFrame)->pd.DataFrame:
    """Return bed coordinates from given dataset."""
    return data.reset_index()[data.index.names]

display(to_bed(epigenomes["promoters"])[:5])

,chrom,chromStart,chromEnd,strand
0,chr1,628964,629220,+
1,chr1,629013,629269,+
2,chr1,629642,629898,+
3,chr1,629847,630103,+
4,chr1,629905,630161,+


In [9]:
genome.bed_to_sequence(to_bed(epigenomes["promoters"])[:2])

array(['AAATGGTCATCCATCCTTTGGCCCCAATACCTAAACTAAGGTCTATGAACAATAAGATGATTTTCTTCAGTGGGACTTTTTTGTTTAATATATTAGATTTGACCTTCAGCAAGGTCAAAGGGAGTCCGAACTAGTCTCAGGCTTCAACATCGAATACGCCGCAGGCCCCTTCGCCCTATTCTTCATAGCCGAATACACAAACATTATTATAATAAACACCCTCACCACTACAATCTTCCTAGGAACAACATATAAC',
       'CAATAAGATGATTTTCTTCAGTGGGACTTTTTTGTTTAATATATTAGATTTGACCTTCAGCAAGGTCAAAGGGAGTCCGAACTAGTCTCAGGCTTCAACATCGAATACGCCGCAGGCCCCTTCGCCCTATTCTTCATAGCCGAATACACAAACATTATTATAATAAACACCCTCACCACTACAATCTTCCTAGGAACAACATATAACGCACTCTCCCCTGAACTCTACACAACATATTTTGTCACCAAGACCCTAC'],
      dtype='<U256')

In [12]:
from keras_bed_sequence import BedSequence

def one_hot_encode(genome:Genome, data:pd.DataFrame, nucleotides:str="actg")->np.ndarray:
    return np.array(BedSequence(
        genome,
        bed=to_bed(data),
        nucleotides=nucleotides,
        batch_size=1
    ))

one_hot_encode(genome, epigenomes["promoters"][:2])

array([[[[0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         ...,
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.]]],


       [[[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         ...,
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.]]]])

In [13]:
def flat_one_hot_encode(genome:Genome, data:pd.DataFrame, window_size:int, nucleotides:str="actg")->np.ndarray:
    return one_hot_encode(genome, data, nucleotides).reshape(-1, window_size*4).astype(int)

flat_one_hot_encode(genome, epigenomes["promoters"][:2], window_size)

array([[0, 1, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 0, 0]])

In [14]:
def to_dataframe(x:np.ndarray, window_size:int, nucleotides:str="actg")->pd.DataFrame:
    return pd.DataFrame(
        x,
        columns = [
            f"{i}{nucleotide}"
            for i in range(window_size)
            for nucleotide in nucleotides
        ]
    )

In [15]:
sequences = {
    region: to_dataframe(
        flat_one_hot_encode(genome, data, window_size),
        window_size
    )
    for region, data in epigenomes.items()
}

In [16]:
sequences["promoters"][:2]
# avevamo una finestra di 200 nucleotidi, ognuno è codifiato one-hot da 4 elementi perché sono 4 le classi (actg)

,0a,0c,0t,0g,1a,1c,1t,1g,2a,2c,...,253t,253g,254a,254c,254t,254g,255a,255c,255t,255g
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0


In [17]:
sequences['enhancers'].to_csv('./pre_processed_dataset/sequence_enhancers.csv')  
sequences["promoters"].to_csv('./pre_processed_dataset/sequence_promoters.csv') 